# Scraping CDC WONDER database

This notebook uses Selenium to query the CDC WONDER database website to get drug overdose mortality rate per county for 2016. It queries 5 states at a time, since querying too many states causes the website to time out.

## Observed dependent variable

In order to get the data for the observed dependent variable, it will be necessary to use Selenium and chromedriver to get these tables from CDC Wonder.

In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import pickle
import time

from bs4 import BeautifulSoup

In [2]:
with open("county_df.pkl", 'rb') as picklefile: # again, rb stands for read binary
    [county_df, states] = pickle.load(picklefile)

In [3]:
states[11] = "Hawaii"
states_by_five = [list(states[i:i+5]) for i in range(0, len(states), 5)]

states_by_five

[['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California'],
 ['Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida'],
 ['Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana'],
 ['Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine'],
 ['Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi'],
 ['Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire'],
 ['New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota'],
 ['Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island'],
 ['South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah'],
 ['Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin'],
 ['Wyoming']]

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time

import os
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)

cdc_wonder_mcd = "https://wonder.cdc.gov/mcd-icd10.html"
driver.get(cdc_wonder_mcd)

In [5]:
agree_button = driver.find_element_by_name("action-I Agree")
agree_button.click()
time.sleep(2)

In [6]:
# Select group by option for results as county
select_group_results = Select(driver.find_element_by_id("SB_1"))
select_group_results.select_by_visible_text("County")

In [7]:
# Check box to get age-adjusted mortality rate
AAR_checkbox = driver.find_element_by_id("CO_aar_enable_label")
AAR_checkbox.click()

In [8]:
def search_dict_values(dic, search_for):
    for key, value in dic.items():
        if search_for in value:
            return key

In [9]:
# Select location. Sometimes it works if all the states are selected, but often the request times out.
# Will select Alabama first.
select_location = Select(driver.find_element_by_id("codes-D77.V9"))

location_dict = {}
for option in select_location.options:
    location_dict[option.get_attribute('value')] = option.text

# let's get the location values for the first 5 states.
location_values = [search_dict_values(location_dict, state) for state in states_by_five[0]]
#location_value = search_dict_values(location_dict, "Alabama")
for location_value in location_values:
    select_location.select_by_value(location_value)

select_location.deselect_by_value("*All*")

In [10]:
# Select year and month; pick 2016
select_year = Select(driver.find_element_by_id("codes-D77.V1"))
select_year.select_by_value("2016")
select_year.deselect_by_value("*All*")

In [11]:
# For underlying cause of death, click radio button for UCD - Drug/Alcohol Induced Causes
drug_alcohol_causes = driver.find_element_by_id("RO_ucdD77.V25")
drug_alcohol_causes.click()

In [12]:
# Now, select all drug induced causes of death.
select_drug_causes = Select(driver.find_element_by_id("codes-D77.V25"))

#for option in select_drug_causes.options:
#    print(option.text)
#    print(option.get_attribute('value'))

select_drug_causes.select_by_value("D")
select_drug_causes.deselect_by_value("*All*")

In [13]:
# Click checkbox to show zero rows.
show_zeros_button = driver.find_element_by_id("CO_show_zeros")
show_zeros_button.click()

# Click checkbox to show suppressed rows.
show_suppressed_button = driver.find_element_by_id("CO_show_suppressed")
show_suppressed_button.click()

In [14]:
# Submit this request to the last send button on the page, so that all of the criteria are included.
send_button = driver.find_element_by_xpath("//div[@class='footer-buttons']/input")
#print(send_button.get_attribute('title'))
send_button.click()

# It takes time for CDC wonder to answer the query and display the results page.
time.sleep(10)

In [15]:
def get_od_mort_df(driver):

# Get the page of results, then get all the tables in the page.
# Get all elements tagged <tr>, and then find the table rows of interest by finding rows containing th class='v'.
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    table = soup.select("tbody")
    table_string = str(table[0])
    table_string_split = table_string.split("<tr>")

    rows = []

    for table_row in table_string_split:
        if 'th class="v"' in table_row:
            rows.append(table_row)

# Split each row by the <td> tag to get a list of each element in the row.
    split_rows = [row.split("<td>") for row in rows]

# Use regular expressions to get rid of all html tags (everything in <>), also commas, tabs and newlines.
    regex = r'(\<.*?\>)|([\n\t,]*)'

    for index, row in enumerate(split_rows):
        split_rows[index] = [re.sub(regex, "", cell) for cell in row]

    cols = ['county_name', 'overdose_deaths', 'population', 'crude_rate', 'age_adj_rate']
    od_mort_df = pd.DataFrame(split_rows, columns=cols)

# The INCITS is located in parentheses at the end of the county_name. It will be easier to join this dataframe
# to the overall dataframe by joining on INCIT, so we'll go ahead and get the INCITS for each row, create a new
# column called INCITS, and then drop the county_name.
    county_names = list(od_mort_df.county_name)
    incits = [county_name.split(" ")[-1] for county_name in county_names]
    incits = [re.sub(r'\(*\)*', "", incit) for incit in incits]
    od_mort_df['INCITS'] = incits
    
    return od_mort_df

In [16]:
od_mort_df = get_od_mort_df(driver)

In [17]:
# After you are done parsing the results, then you can click back to the request page in order to
# re-run the request, except this time you are going to pick a different set of states, and also
# de-select the states that you just queried. Query 5 states at a time.
request_tab = driver.find_element_by_name("tab-request")
request_tab.click()
time.sleep(5)

## Start loop to finish queries for all states

Now that the appropriate dataframe has been started, we can loop through states_by_five and finish the queries for all of the other 45 states. We will temporarily store the results for these tables in a temporary dataframe, and then concatenate it to the original od_mort_df.

In [19]:
for i in range(1, len(states_by_five)):

    select_location = Select(driver.find_element_by_id("codes-D77.V9"))

    # deselect all the previous location values
    for location_value in location_values:
        select_location.deselect_by_value(location_value)

    # Get a new set of location values.
    location_values = [search_dict_values(location_dict, state) for state in states_by_five[i]]

    for location_value in location_values:
        select_location.select_by_value(location_value)
        
    # Submit this request to the last send button on the page, so that all of the criteria are included.
    send_button = driver.find_element_by_xpath("//div[@class='footer-buttons']/input")
    send_button.click()
    
    time.sleep(10)
    
    temp_od_mort_df = get_od_mort_df(driver)
    
    od_mort_df = pd.concat([od_mort_df, temp_od_mort_df], ignore_index=True)
    
    # After you are done parsing the results, then you can click back to the request page in order to
    # re-run the request, except this time you are going to pick a different set of states, and also
    # de-select the states that you just queried. Query 5 states at a time.
    request_tab = driver.find_element_by_name("tab-request")
    request_tab.click()
    time.sleep(5)

In [20]:
driver.close()

In [21]:
od_mort_df.head()

,county_name,overdose_deaths,population,crude_rate,age_adj_rate,INCITS
0,Autauga County AL (01001),Suppressed,55416,Suppressed,Suppressed,01001
1,Baldwin County AL (01003),34,208563,16.3,15.7,01003
2,Barbour County AL (01005),Suppressed,25965,Suppressed,Suppressed,01005
3,Bibb County AL (01007),Suppressed,22643,Suppressed,Suppressed,01007
4,Blount County AL (01009),15,57704,Unreliable,Unreliable,01009


In [22]:
# save od_mort_df dataframe to pickle.
#with open("od_mort_df.pkl", "wb") as picklefile:
#    pickle.dump(od_mort_df, picklefile)

In [23]:
#with open("od_mort_df.pkl", 'rb') as picklefile: # again, rb stands for read binary
#    od_mort_df = pickle.load(picklefile)

In [24]:
#od_mort_df.shape --> # shape without suppressed and zero rows is (1000, 6)

In [27]:
print(od_mort_df.shape) # you have a difference of 5 counties; you should compare INCITS to figure out what's different.
print(county_df.shape)

(3147, 6)
(3142, 3)


In [26]:
# save od_mort_df dataframe to pickle.
with open("od_mort_df.pkl", "wb") as picklefile:
    pickle.dump(od_mort_df, picklefile)

In [33]:
od_mort_df.head()

,county_name,overdose_deaths,population,crude_rate,age_adj_rate,INCITS
0,Autauga County AL (01001),Suppressed,55416,Suppressed,Suppressed,01001
1,Baldwin County AL (01003),34,208563,16.3,15.7,01003
2,Barbour County AL (01005),Suppressed,25965,Suppressed,Suppressed,01005
3,Bibb County AL (01007),Suppressed,22643,Suppressed,Suppressed,01007
4,Blount County AL (01009),15,57704,Unreliable,Unreliable,01009


In [36]:
set(county_df.INCITS).difference(set(od_mort_df.INCITS))

{'02158', '46102'}

In [37]:
county_df[county_df.INCITS == '02158']

,INCITS,county_name,state
92,02158,Kusilvak Census Area[10],Alaska


In [38]:
county_df[county_df.INCITS == '46102']

,INCITS,county_name,state
2500,46102,Oglala Lakota County,South Dakota
